# Chapter 02: Tools

LLMs use various tools to achieve specific goals, streamline operations, and automate tasks. These tools include:

1. **Data retrieval tools:** Extract information from systems or databases using APIs, SDKs, and real-time metrics.
2. **Communication tools:** Facilitate data exchange with external stakeholders via emails, notifications, or alerts.
3. **Data manipulation tools:** Update or modify data within systems, often requiring approval to manage operational impacts.

Additional tools also exist to handle tasks LLMs struggle with, like performing calculations or accessing current date and time.

In [1]:
import re
import numexpr as ne
import dirtyjson as json
from datetime import datetime
from typing import Any, Callable
from pydantic import BaseModel, Field, ValidationError
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=250,
    temperature=0.2,
)

To allow LLMs to leverage tools effectively, a few steps are needed. First, we need to communicate to the LLM that it has access to specific tools by providing the tool's name, a description of when or why the tool should be used, and the input arguments required for its successful execution. For our series on LLM-powered AI Agents, we will use Pydantic to simplify this process. When an LLM provides the input arguments, we need to validate them, execute the tool if the inputs are correct, and show the tool's output to the LLM. If the inputs are incorrect, we need to inform the LLM of the mistake so it can correct and resubmit the input.

Tools should be represented in a format such as: Tool Name: ..., Tool Description: ..., Tool Input: ...

In [4]:
class Tool(BaseModel):
    """Class that implements a tool.

    Attributes:
        function: The function that will be invoked when calling this tool.
        name: The name of the tool.
        description: The description of when to use the tool or what the tool does.
        args_schema: The Pydantic model that represents the input arguments.
    """

    function: Callable[[Any], Any]
    name: str
    description: str
    args_schema: type[BaseModel] | None = None

    @property
    def args(self) -> dict[str, Any]:
        """Gets the tool model JSON schema."""
        if self.args_schema is None:
            return {}
        return self.args_schema.model_json_schema()

    def parse_input(self, tool_input: dict[str, Any]) -> dict[str, Any]:
        """Converts tool input to pydantic model."""
        input_args = self.args_schema
        if input_args is not None:
            result = input_args.model_validate(tool_input)
            return {key: getattr(result, key) for key, _ in result.model_dump().items() if key in tool_input}
        return tool_input

    def invoke(self, tool_input: dict[str, Any]) -> Any:
        """Invokes a tool given arguments provided by an LLM."""
        try:
            parsed_input = self.parse_input(tool_input)
            if not parsed_input:
                output = self.function()
            else:
                output = self.function(**parsed_input)
        except (ValidationError, TypeError) as error:
            output = "\n\n".join(
                [
                    f"Could not run tool {self.name} with input:\n{tool_input}",
                    f"The error was:\n{error}",
                    "You need to correct your response",
                    f"Your <input of the tool to use> must be a JSON format with the keyword arguments of:\n{self.args}",
                ]
            )
        return output

    def __str__(self) -> str:
        return f"- Tool Name: {self.name}, Tool Description: {self.description}, Tool Input: {self.args}"

In this example, two tools are defined to be used by an LLM: 
- **A calculator tool**: The calculator tool evaluates mathematical expressions provided as strings, with an accompanying Pydantic model to validate the input.
- **A current date tool**: The current date tool provides the current local date and time. 

For each tool, a name, description, function, and, if needed, argument schema are provided.

In [5]:
class Calculator(BaseModel):
    expression: str = Field(description="A math expression")

calculator = Tool(
    function=lambda expression: ne.evaluate(expression).item(),
    name="Calculator",
    description="Use this tool when you want to do calculations",
    args_schema=Calculator,
)

print(calculator)

- Tool Name: Calculator, Tool Description: Use this tool when you want to do calculations, Tool Input: {'properties': {'expression': {'description': 'A math expression', 'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'title': 'Calculator', 'type': 'object'}


In [6]:
current_date = Tool(
    function=lambda: datetime.now(),
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

print(current_date)

- Tool Name: Current Date, Tool Description: Use this tool to access the current local date and time, Tool Input: {}


In this setup, the `system_prompt` outlines how the LLM should structure its responses when addressing a user query. This includes specifying how and when to use the defined tools to solve the problems presented by users. When the LLM encounters a user prompt that requires a calculation or a specific operation, it must adhere to a predefined structure in its response:

1. **Thought**: The LLM first generates a thought, which is an explanation or reasoning about what steps need to be taken to solve the problem.
2. **Tool**: Based on the thought, the LLM selects the appropriate tool that is designed to perform the necessary operation.
3. **Tool Input**: After selecting the tool, the LLM decides on the correct input needed for the tool to successfully complete the task or subtask.

The order of thought, tool, and tool input is crucial. This is because LLMs work by predicting the next token with the highest probability based on the previous tokens. If the context is well-defined (i.e., the thought is clearly articulated), the LLM can more accurately choose the appropriate tool. Subsequently, given the tool and the thought, the LLM can then determine the precise input required for the tool to function correctly.

In [7]:
tools = [calculator, current_date]
tools_str = "\n\n".join([str(tool) for tool in tools])

system_prompt = f"""You are an AI assistant designed to help users with a variety of tasks.

### Tools ###

You have access to the following tools:
{tools_str}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>"""

tools_map = {tool.name: tool for tool in tools}

prompt = "Calculate the total raw cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12."

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

Thought: The user wants to calculate the sum of several numbers. I can use the Calculator tool for this task.

Tool: Calculator

Tool Input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}


The function `extract_tool_use(output: str)` is used to parse a structured response from an LLM to extract three key components: the thought process, the name of the tool to be used, and the tool's input. It uses a regular expression pattern to match these components in the output string. If the pattern does not match, it raises a ValueError with guidance on the correct response format. When a match is found, it retrieves and trims the thought, tool, and tool input from the matched groups. This function ensures that the LLM's response adheres to the expected format and extracts necessary details for further processing.

In [8]:
def extract_tool_use(output: str) -> tuple[str, str, str]:
    pattern = r"\s*Thought: (.*?)\n+Tool: ([a-zA-Z0-9_ ]+).*?\n+Tool Input: .*?(\{.*\})"

    match = re.search(pattern, output, re.DOTALL)
    if not match:
        raise ValueError(
            f"You made a mistake in your response: {output}\n\n"
            + f"You need to correct your response\n\n"
            + "You should respond with:\n```\nThought: <thought process on how to respond to the prompt>\n\nTool: <name of the tool to use>\n\nTool Input: <input of the tool to use>\n```"
        )

    thought = match.group(1).strip()
    tool = match.group(2).strip()
    tool_input = match.group(3).strip()
    return thought, tool, tool_input

In [9]:
try:
    thought, tool, tool_input = extract_tool_use(output)
    print(f"Thought: {thought}")
    print(f"Tool: {tool}")
    print(f"Tool Input: {tool_input}")
except ValueError as error:
    print(error)

Thought: The user wants to calculate the sum of several numbers. I can use the Calculator tool for this task.
Tool: Calculator
Tool Input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}


Next, the JSON representation of the tool input must be converted into a dictionary to enable the tool's invocation.

In [10]:
tool_input = dict(json.loads(tool_input))
tool = tools_map.get(tool)
tool_output = tool.invoke(tool_input)
print(tool_output)

1289.98


In this scenario, the `system_prompt` defines how an LLM should handle user queries, detailing step-by-step usage of available tools and structured response formats. It includes guidance on composing responses when using tools as well as when delivering the final answer. The `prompt` presented to the LLM is a request to calculate the total raw cost of a series of amounts, augmented by the LLM's previous work, including usage of the "Calculator" tool. Finally, the LLM generates the final answer, following the provided instructions and prompt.

In [11]:
system_prompt = f"""You are an AI assistant designed to help users with a variety of tasks.

### Tools ###

You have access to the following tools:
{tools_str}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>

When you know the final answer to the user's query you should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query"""

prompt = f"""Calculate the total raw cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12.

This was your previous work:

{output}

Observation: Tool Output: {tool_output}"""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])

print(output)

Thought: The tool has successfully calculated the sum of the given numbers. I can now provide the user with the result.

Final Answer: The total raw cost is $1289.98.


The function `extract_final_answer(output: str)` is used to parse a structured response from an LLM to extract two key components: the thought process and the final answer.

In [12]:
def extract_final_answer(output: str) -> tuple[str, str]:
    pattern = r"\s*Thought: (.*?)\n+Final Answer:([\s\S]*.*?)(?:$)"

    match = re.search(pattern, output, re.DOTALL)
    if not match:
        raise ValueError(
            f"You made a mistake in your response: {output}\n\n"
            + f"Your need to correct your response\n\n"
            + "You should respond with:\n```\nThought: <thought process on how to respond to the prompt>\n\nFinal Answer: <response to the prompt>\n```"
        )

    thought = match.group(1).strip()
    final_answer = match.group(2).strip()
    return thought, final_answer

In [13]:
try:
    thought, final_answer = extract_final_answer(output)
    print(f"Thought: {thought}")
    print(f"Final Answer: {final_answer}")
except ValueError as error:
    print(error)

Thought: The tool has successfully calculated the sum of the given numbers. I can now provide the user with the result.
Final Answer: The total raw cost is $1289.98.


Now, let's do the same thing to retrieve the current date.

In [14]:
system_prompt = f"""You are an AI assistant designed to help users with a variety of tasks.

### Tools ###

You have access to the following tools:
{tools_str}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>

When you know the final answer to the user's query you should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query"""

prompt = "What day do we have?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])

print(output)

Thought: The user is asking for the current date. I can use the "Current Date" tool to provide this information.

Tool: Current Date

Tool Input: {}


In [15]:
try:
    thought, tool, tool_input = extract_tool_use(output)
    print(f"Thought: {thought}")
    print(f"Tool: {tool}")
    print(f"Tool Input: {tool_input}")
except ValueError as error:
    print(error)

Thought: The user is asking for the current date. I can use the "Current Date" tool to provide this information.
Tool: Current Date
Tool Input: {}


In [16]:
tool_input = dict(json.loads(tool_input))
tool = tools_map.get(tool)
tool_output = tool.invoke(tool_input)
print(tool_output)

2024-08-16 13:56:44.245589


In [17]:
system_prompt = f"""You are an AI assistant designed to help users with a variety of tasks.

### Tools ###

You have access to the following tools:
{tools_str}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>

When you know the final answer to the user's query you should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query"""

prompt = f"""What day do we have?

This was your previous work:

{output}

Observation: Tool Output: {tool_output}"""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])

print(output)

Thought: The tool has provided the current date and time. The user only asked for the day, so I will extract that information from the date.

Final Answer: Today is August 16, 2024.


In [18]:
try:
    thought, final_answer = extract_final_answer(output)
    print(f"Thought: {thought}")
    print(f"Final Answer: {final_answer}")
except ValueError as error:
    print(error)

Thought: The tool has provided the current date and time. The user only asked for the day, so I will extract that information from the date.
Final Answer: Today is August 16, 2024.


In summary, it is clear that when an LLM doesn't have access to tools, we can use single completion prompts to obtain desired results. Conversely, if the LLM has tools, employing Chain-of-Thought prompting, particularly the ReAct prompting method, and requesting structured outputs that can be parsed using regular expressions becomes necessary. While outputs can be formatted in YAML or JSON, expressing outputs as plain text is more cost-effective.